In [1]:
import pandas as pd
from  sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

input_file = 'diabetes.csv'
df = pd.read_csv(input_file)

In [2]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
df.astype(bool).sum(axis=0)

Pregnancies                 657
Glucose                     763
BloodPressure               733
SkinThickness               541
Insulin                     394
BMI                         757
DiabetesPedigreeFunction    768
Age                         768
Outcome                     268
dtype: int64

In [6]:
#Glucose can not be 0. BP can not be zero. Insulin can not be zero, BMI can not be zero
df.shape

(768, 9)

In [10]:
X = df[list(df.columns)[:-1]] 
y = df['Outcome']
OX_train, OX_test, Oy_train, Oy_test = train_test_split(X, y, test_size=0.2)

In [21]:
k_fold = KFold(n_splits=10)
lr_model = LogisticRegression()
scores = []
OX_train.reset_index(drop=True, inplace=True)
Oy_train.reset_index(drop=True, inplace=True)
print(OX_train.head())
for train_index, test_index in k_fold.split(OX_train):
    X_train, X_test, y_train, y_test = OX_train.iloc[train_index,:], OX_train.iloc[test_index,:], Oy_train[train_index], Oy_train[test_index]
    lr_model.fit(X_train, y_train)
    lr_model.predict(X_test)
    scores.append(lr_model.score(X_test, y_test))
print(scores)


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1       93             56             11        0  22.5   
1            2       96             68             13       49  21.1   
2            3       99             62             19       74  21.8   
3           10       68            106             23       49  35.5   
4            1       71             78             50       45  33.2   

   DiabetesPedigreeFunction  Age  
0                     0.417   22  
1                     0.647   26  
2                     0.279   26  
3                     0.285   47  
4                     0.422   21  
[0.7258064516129032, 0.6774193548387096, 0.7580645161290323, 0.7419354838709677, 0.8360655737704918, 0.7704918032786885, 0.7868852459016393, 0.7540983606557377, 0.8852459016393442, 0.8032786885245902]


/home/rakesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/rakesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/rakesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/rakesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/rakesh/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Spec

In [23]:
predict = lr_model.predict(OX_test)

In [24]:
print(predict)

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 1 0 1]


In [26]:
lr_model.score(OX_test, Oy_test)

0.7857142857142857